<a href="https://colab.research.google.com/github/Chava-Sai/Translators_Comparsion/blob/main/Ai4Bharat_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AI4 Bharath Translator

In [ ]:
# Check if GPU is available
import tensorflow as tf
if tf.test.gpu_device_name():
    print("GPU is available")
    # Explicitly set a GPU device if multiple GPUs are available
    # For example, use the first GPU (device: 0)
    # You can change the device number as needed
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("GPU is not available")

# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

GPU is available
Cloning into 'indicTrans'...
remote: Enumerating objects: 700, done.
remote: Counting objects: 100% (403/403), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 700 (delta 280), reused 341 (delta 238), pack-reused 297
Receiving objects: 100% (700/700), 2.64 MiB | 6.57 MiB/s, done.
Resolving deltas: 100% (407/407), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1399, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 1399 (delta 135), reused 147 (delta 120), pack-reused 1219
Receiving objects: 100% (1399/1399), 9.57 MiB | 7.18 MiB/s, done.
Resolving deltas: 100% (745/745), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/

In [ ]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install ./
! pip install xformers
%cd ..

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49170 sha256=e94294ddcc346fdb987f4247e01b184eaec80690fdae20988017dfe3394c0d16
  Stored in directory: /root/.cache/pip/wheels/80/d8/15/4c5ebbe883513f003cb055a0369c77c9df857023a706f39e70
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl s

In [ ]:
# add fairseq folder to python path
import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"
# sanity check to see if fairseq is installed
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

This is just a warning: triton is not available


In [ ]:
# download the indictrans model
# downloading the indic-en model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip
!unzip indic2en.zip

# downloading the en-indic model
# !wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip
# !unzip en2indic.zip

# # downloading the indic-indic model
# !wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/m2m.zip
# !unzip m2m.zip
%cd indicTrans

--2024-03-02 08:51:43--  https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip
Resolving ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)... 164.52.206.155, 164.52.210.96, 101.53.152.33, ...
Connecting to ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)|164.52.206.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4759117228 (4.4G) [application/zip]
Saving to: ‘indic2en.zip’

indic2en.zip        100%[===================>]   4.43G  11.7MB/s    in 6m 35s  

2024-03-02 08:58:20 (11.5 MB/s) - ‘indic2en.zip’ saved [4759117228/4759117228]

Archive:  indic2en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/prep

In [ ]:
from indicTrans.inference.engine import Model

indic2en_model = Model(expdir='../indic-en')

Initializing vocab and bpe
Initializing model for translation


In [ ]:
# Function to transliterate Hindi text to English
def translate_to_english(text):
    if pd.isnull(text) or not isinstance(text, str):
        return ''  # or any default value
    english_text = indic2en_model.translate_paragraph(text, 'hi', 'en')
    return english_text

In [ ]:
import pandas as pd
from tqdm import tqdm
df = pd.read_excel('/content/Translator_1L-2L_Data.xlsx')
df = df.head(50000)

In [ ]:
df['ai4_english']=None
for n, text in enumerate(tqdm(df['hindi'])):
  df['ai4_english'][n] = translate_to_english(text)
  # break
df.head(5)

Streaming output truncated to the last 5000 lines.

100%|██████████| 1/1 [00:00<00:00, 5295.84it/s]

100%|██████████| 1/1 [00:00<00:00, 4911.36it/s]

100%|██████████| 1/1 [00:00<00:00, 5777.28it/s]

100%|██████████| 1/1 [00:00<00:00, 5533.38it/s]

100%|██████████| 1/1 [00:00<00:00, 4341.93it/s]

100%|██████████| 1/1 [00:00<00:00, 4319.57it/s]

100%|██████████| 1/1 [00:00<00:00, 5667.98it/s]

100%|██████████| 1/1 [00:00<00:00, 5652.70it/s]

100%|██████████| 1/1 [00:00<00:00, 5102.56it/s]

100%|██████████| 1/1 [00:00<00:00, 4447.83it/s]

100%|██████████| 1/1 [00:00<00:00, 4644.85it/s]

100%|██████████| 1/1 [00:00<00:00, 2970.47it/s]

100%|██████████| 1/1 [00:00<00:00, 5526.09it/s]

100%|██████████| 1/1 [00:00<00:00, 4490.69it/s]

100%|██████████| 1/1 [00:00<00:00, 4848.91it/s]

100%|██████████| 1/1 [00:00<00:00, 5433.04it/s]

100%|██████████| 1/1 [00:00<00:00, 4219.62it/s]

100%|██████████| 1/1 [00:00<00:00, 4826.59it/s]

100%|██████████| 1/1 [00:00<00:00, 3971.88it/s]

100%|██████████| 

In [ ]:
df.to_excel('Head(50k)_1L-2L_Ai_pro_Trans.xlsx')
from google.colab import files
files.download('Head(50k)_1L-2L_Ai_pro_Trans.xlsx')

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
from tqdm import tqdm
df = pd.read_excel('/content/Translator_1L-2L_Data.xlsx')
df = df.tail(50000)

In [ ]:
df['ai4_english']=None
for n, text in enumerate(tqdm(df['hindi'])):
  df['ai4_english'][n] = translate_to_english(text)
  # break
df.head(5)

Streaming output truncated to the last 5000 lines.
100%|██████████| 1/1 [00:00<00:00, 3890.82it/s]

100%|██████████| 1/1 [00:00<00:00, 5165.40it/s]

100%|██████████| 1/1 [00:00<00:00, 4951.95it/s]

100%|██████████| 1/1 [00:00<00:00, 4583.94it/s]

100%|██████████| 1/1 [00:00<00:00, 5140.08it/s]

100%|██████████| 1/1 [00:00<00:00, 3404.47it/s]

100%|██████████| 1/1 [00:00<00:00, 5570.12it/s]

100%|██████████| 1/1 [00:00<00:00, 5035.18it/s]

100%|██████████| 1/1 [00:00<00:00, 4911.36it/s]

100%|██████████| 1/1 [00:00<00:00, 5890.88it/s]

100%|██████████| 1/1 [00:00<00:00, 5461.33it/s]

100%|██████████| 1/1 [00:00<00:00, 6269.51it/s]

100%|██████████| 1/1 [00:00<00:00, 5924.16it/s]

100%|██████████| 1/1 [00:00<00:00, 6533.18it/s]

100%|██████████| 1/1 [00:00<00:00, 5454.23it/s]

100%|██████████| 1/1 [00:00<00:00, 5890.88it/s]

100%|██████████| 1/1 [00:00<00:00, 6223.00it/s]

100%|██████████| 1/1 [00:00<00:00, 4249.55it/s]

100%|██████████| 1/1 [00:00<00:00, 5957.82it/s]

100%|██████████| 1


100%|██████████| 1/1 [00:00<00:00, 2892.62it/s]


100%|██████████| 1/1 [00:00<00:00, 5356.71it/s]

100%|██████████| 1/1 [00:00<00:00, 2837.82it/s]


100%|██████████| 1/1 [00:00<00:00, 6241.52it/s]

100%|██████████| 1/1 [00:00<00:00, 5262.61it/s]

100%|██████████| 1/1 [00:00<00:00, 3017.48it/s]


100%|██████████| 1/1 [00:00<00:00, 5874.38it/s]

100%|██████████| 1/1 [00:00<00:00, 3088.59it/s]


...

100%|██████████| 1/1 [00:00<00:00, 2870.84it/s]


100%|██████████| 2/2 [00:00<00:00, 4004.11it/s]


100%|██████████| 1/1 [00:00<00:00, 4940.29it/s]

100%|██████████| 1/1 [00:00<00:00, 5526.09it/s]

100%|██████████| 1/1 [00:00<00:00, 5555.37it/s]

100%|██████████| 1/1 [00:00<00:00, 5957.82it/s]

100%|██████████| 1/1 [00:00<00:00, 5302.53it/s]

100%|██████████| 1/1 [00:00<00:00, 5356.71it/s]

100%|██████████| 1/1 [00:00<00:00, 5841.65it/s]

100%|██████████| 1/1 [00:00<00:00, 3898.05it/s]

100%|██████████| 1/1 [00:00<00:00, 5426.01it/s]

100%|██████████| 1/1 [00:00<00:00, 4410.41it/s]

100%|██████████| 1/1 [00:00<00:00, 5053.38it/s]

100%|██████████| 1/1 [00:00<00:00, 5349.88it/s]

100%|██████████| 1/1 [00:00<00:00, 4804.47it/s]

100%|██████████| 1/1 [00:00<00:00, 4702.13it/s]

100%|██████████| 1/1 [00:00<00:00, 5047.30it/s]

100%|██████████| 1/1 [00:00<00:00, 4249.55it/s]

100%|██████████| 1/1 [00:00<00:00, 4686.37it/s]

100%|██████████| 1/1 [00:00<00:00, 5053.38it/s]

100%|██████████| 1/1 [00:00<00:00, 5584.96it/s]

100%|██████████| 1/1 [00:00<00:00, 3826.92it/s]

100%|██████████| 1/1

100%|██████████| 1/1 [00:00<00:00, 4728.64it/s]

100%|██████████| 2/2 [00:00<00:00, 4130.28it/s]


100%|██████████| 1/1 [00:00<00:00, 4271.19it/s]

100%|██████████| 1/1 [00:00<00:00, 4639.72it/s]

100%|██████████| 1/1 [00:00<00:00, 5282.50it/s]

100%|██████████| 1/1 [00:00<00:00, 5614.86it/s]

100%|██████████| 1/1 [00:00<00:00, 1752.74it/s]

100%|██████████| 1/1 [00:00<00:00, 5322.72it/s]

100%|██████████| 1/1 [00:00<00:00, 4559.03it/s]

100%|██████████| 1/1 [00:00<00:00, 4905.62it/s]

100%|██████████| 1/1 [00:00<00:00, 4848.91it/s]

100%|██████████| 1/1 [00:00<00:00, 5833.52it/s]

100%|██████████| 1/1 [00:00<00:00, 5356.71it/s]

100%|██████████| 1/1 [00:00<00:00, 5210.32it/s]

100%|██████████| 1/1 [00:00<00:00, 6078.70it/s]

100%|██████████| 1/1 [00:00<00:00, 5223.29it/s]

100%|██████████| 1/1 [00:00<00:00, 5035.18it/s]

100%|██████████| 1/1 [00:00<00:00, 5005.14it/s]

100%|██████████| 1/1 [00:00<00:00, 5256.02it/s]

100%|██████████| 2/2 [00:00<00:00, 10180.35it/s]

100%|██████████| 1/1 [00:00<00:00, 4917.12it/s]

100%|██████████| 1/1 [00:00<00:00, 6017.65it/s]

100%|██████████| 1/

In [ ]:
df.to_excel('Tail(50k)_1L-2L_Ai_pro_Trans.xlsx')
from google.colab import files
files.download('Tail(50k)_1L-2L_Ai_pro_Trans.xlsx')

In [ ]:
import pandas as pd
from tqdm import tqdm
df = pd.read_excel('/content/Translator_2L-3L_Data.xlsx')
df = df.head(50000)

In [ ]:
df['ai4_english']=None
for n, text in enumerate(tqdm(df['hindi'])):
  df['ai4_english'][n] = translate_to_english(text)
  # break
df.head(5)

In [ ]:
df.to_excel('Head(50k)_2L-3L_Ai_pro_Trans.xlsx')
from google.colab import files
files.download('Head(50k)_2L-3L_Ai_pro_Trans.xlsx')

In [ ]:
import pandas as pd
from tqdm import tqdm
df = pd.read_excel('/content/Translator_2L-3L_Data.xlsx')
df = df.head(50000)

In [ ]:
df['ai4_english']=None
for n, text in enumerate(tqdm(df['hindi'])):
  df['ai4_english'][n] = translate_to_english(text)
  # break
df.head(5)

In [ ]:
df.to_excel('Tail(50k)_2L-3L_Ai_pro_Trans.xlsx')
from google.colab import files
files.download('Tail(50k)_2L-3L_Ai_pro_Trans.xlsx')

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess(text):
    tokenizer = RegexpTokenizer(r'\w+')
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    words = tokenizer.tokenize(str(text))
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalpha()]
    words = [word for word in words if word not in stop_words]

    print(words)  # Add this line to print the tokenized words

    return ' '.join(words)

def sentence_similarity(sentence1, sentence2):
    preprocessed_sentence1 = preprocess(sentence1)
    preprocessed_sentence2 = preprocess(sentence2)

    # Check if vocabulary is empty
    if not preprocessed_sentence1 or not preprocessed_sentence2:
        return 0.0  # Return 0 similarity for empty vocabulary

    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([preprocessed_sentence1, preprocessed_sentence2])

    similarity = cosine_similarity(vectors[0], vectors[1])[0][0]
    return similarity

# Assuming you have a DataFrame named 'df' with 'english' and 'eng_trans' columns
# If not, replace 'df' with your actual DataFrame

# Load the dataset
df = pd.read_excel('Google_Trans.xlsx')  # Replace 'your_dataset.csv' with your actual file path

# Print unique words
unique_words = set()
df.apply(lambda row: unique_words.update(preprocess(row['english']).split()), axis=1)
df.apply(lambda row: unique_words.update(preprocess(row['eng_trans']).split()), axis=1)
print("Unique Words:", unique_words)

# Add a new column 'sts_score_eng_trans'
df['sts_score_Google_Trans_text'] = df.apply(lambda row: sentence_similarity(row['english'], row['eng_trans']), axis=1)

# Round the STS scores to 4 decimals
df['sts_score_Google_Trans_text'] = df['sts_score_Google_Trans_text'].round(4)

# Save the updated dataset to a new Excel file
df.to_excel('Score_Google_Trans_text.xlsx', index=False)
files.download('Score_Google_Trans_text.xlsx')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['give', 'application', 'accessibility', 'workout']
['accerciser', 'accessibility', 'explorer']
['default', 'plugin', 'layout', 'bottom', 'panel']
['default', 'plugin', 'layout', 'top', 'panel']
['list', 'plugins', 'disabled', 'default']
['highlight', 'duration']
['duration', 'highlight', 'box', 'selecting', 'accessible', 'node']
['highlight', 'border', 'color']
['color', 'opacity', 'highlight', 'border']
['highlight', 'fill', 'color']
['color', 'opacity', 'highlight', 'fill']
['api', 'browser']
['browse', 'various', 'method', 'current', 'accessible']
['hide', 'private', 'attribute']
['method']
['property']
['value']
['ipython', 'console']
['interactive', 'console', 'manipulating', 'currently', 'selected', 'accessible']
['event', 'monitor']
['monitor', 'event']
['c', 'lear', 'selection']
['everything']
['selected', 'application']
['selected', 'accessible']
['source']
['event', 'monitor']
['show', 'event', 'occur', 'selected', 'type', 'source']
['highlight', 'last', 'event', 'entry']
['

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess(text):
    tokenizer = RegexpTokenizer(r'\w+')
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    words = tokenizer.tokenize(str(text))
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalpha()]
    words = [word for word in words if word not in stop_words]

    print(words)  # Add this line to print the tokenized words

    return ' '.join(words)

def sentence_similarity(sentence1, sentence2):
    preprocessed_sentence1 = preprocess(sentence1)
    preprocessed_sentence2 = preprocess(sentence2)

    # Check if vocabulary is empty
    if not preprocessed_sentence1 or not preprocessed_sentence2:
        return 0.0  # Return 0 similarity for empty vocabulary

    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([preprocessed_sentence1, preprocessed_sentence2])

    similarity = cosine_similarity(vectors[0], vectors[1])[0][0]
    return similarity

# Assuming you have a DataFrame named 'df' with 'english' and 'eng_trans' columns
# If not, replace 'df' with your actual DataFrame

# Load the dataset
df = pd.read_excel('Ai_pro_Trans.xlsx')  # Replace 'your_dataset.csv' with your actual file path

# Print unique words
unique_words = set()
df.apply(lambda row: unique_words.update(preprocess(row['english']).split()), axis=1)
df.apply(lambda row: unique_words.update(preprocess(row['ai4_english']).split()), axis=1)
print("Unique Words:", unique_words)

# Add a new column 'sts_score_eng_trans'
df['sts_score_ai4_english_text'] = df.apply(lambda row: sentence_similarity(row['english'], row['ai4_english']), axis=1)

# Round the STS scores to 4 decimals
df['sts_score_ai4_english_text'] = df['sts_score_ai4_english_text'].round(4)

# Save the updated dataset to a new Excel file
df.to_excel('Final_AI_Pro_Output.xlsx', index=False)  # Replace 'your_updated_dataset.csv' with your desired file name


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['give', 'application', 'accessibility', 'workout']
['accerciser', 'accessibility', 'explorer']
['default', 'plugin', 'layout', 'bottom', 'panel']
['default', 'plugin', 'layout', 'top', 'panel']
['list', 'plugins', 'disabled', 'default']
['highlight', 'duration']
['duration', 'highlight', 'box', 'selecting', 'accessible', 'node']
['highlight', 'border', 'color']
['color', 'opacity', 'highlight', 'border']
['highlight', 'fill', 'color']
['color', 'opacity', 'highlight', 'fill']
['api', 'browser']
['browse', 'various', 'method', 'current', 'accessible']
['hide', 'private', 'attribute']
['method']
['property']
['value']
['ipython', 'console']
['interactive', 'console', 'manipulating', 'currently', 'selected', 'accessible']
['event', 'monitor']
['monitor', 'event']
['c', 'lear', 'selection']
['everything']
['selected', 'application']
['selected', 'accessible']
['source']
['event', 'monitor']
['show', 'event', 'occur', 'selected', 'type', 'source']
['highlight', 'last', 'event', 'entry']
['

In [ ]:
files.download('Final_AI_Pro_Output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.head(5)

,Unnamed: 0,hindi,english,eng_trans,ai4_english,sts_score_Google_Trans_text,sts_score_ai4_english_text
0,0,"और चीजों के दूसरे पक्ष पर,","And on the other side of things,","And on the other side of things,","and on the other side of things,",1.0000,1.0000
1,1,"अगर आप एक वस्तु रखना चाहते हैं,","if you want to keep an object moving,","If you want to keep an object,","If you want to keep one thing,",0.7765,0.3361
2,2,आप इसे करने के लिए एक ताकत लगाने रखने के लिए क...,you have to keep applying a force to it.,You have to keep a strength to do it.,You have to keep putting a force to it.,0.2606,0.5031
3,3,हम हमारे रोजमर्रा के अनुभव में कभी नहीं किया है,We 've never in our everyday experience,We have never done in our everyday experience,We have never done it in our everyday experience,0.7765,0.7765
4,4,एक वस्तु है कि बस चलती रहती देखा,seen an object that just keeps moving,There is an object that just saw the bus walking,There is an object that saw the bus moving,0.1444,0.3361


In [ ]:
from google.colab import files
files.download('Google_Trans.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Replace 'your_data.csv' with the path to your CSV file
file_path = 'Score_Google_Trans_text.xlsx'

# Load data from CSV file into a pandas DataFrame
df = pd.read_excel(file_path)

# Count occurrences where sts_score_Google_Trans_text > sts_score_ai4_english_text
greater_than_count = (df['sts_score_Google_Trans_text'] > df['sts_score_ai4_english_text']).sum()

# Count occurrences where sts_score_Google_Trans_text < sts_score_ai4_english_text
less_than_count = (df['sts_score_Google_Trans_text'] < df['sts_score_ai4_english_text']).sum()

# Count occurrences where sts_score_Google_Trans_text == sts_score_ai4_english_text
equal_to_count = (df['sts_score_Google_Trans_text'] == df['sts_score_ai4_english_text']).sum()

print(f"Google_Trans Better: {greater_than_count}")
print(f"Ai4bharat Trans Better: {less_than_count}")
print("--------------------------------------------")
print(f"Both Are Equal: {equal_to_count}")


Google_Trans Better: 58
Ai4bharat Trans Better: 436
--------------------------------------------
Both Are Equal: 506
